<a href="https://colab.research.google.com/github/drf0rk/AnxLight/blob/main/notebook/AnxLight_Launcher_v0.0.9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Cell 1: AnxLight v3 Pre-Flight Setup (Unified Log)
# This cell ensures the repository is up-to-date and runs the main environment setup with enhanced, unified logging.

import os
import subprocess
import sys
import time
import platform
from datetime import datetime

# --- Unified Logging Setup ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f"/content/anxlight_logs_{timestamp}"
os.makedirs(log_dir, exist_ok=True)
# Use a single .txt log file for the entire session
session_log_file = f"{log_dir}/session_log_{timestamp}.txt"
os.environ['ANXLIGHT_LOG_FILE'] = session_log_file # Share path with Cell 2

def log_message(message, level="INFO"):
    """Log a message to both console and the unified session log file"""
    timestamp_msg = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_msg = f"[{timestamp_msg}] [{level}] {message}"
    print(formatted_msg)
    with open(session_log_file, "a") as f:
        f.write(formatted_msg + "\n")

# --- Initial System Checks ---
log_message(f"--- CELL 1: PRE-FLIGHT SETUP START ---")
log_message(f"Python version: {sys.version}")
log_message(f"Platform: {platform.platform()}")

try:
    git_version = subprocess.check_output(["git", "--version"], text=True).strip()
    log_message(f"Git version: {git_version}")
except Exception as e:
    log_message(f"Git not found or not working properly: {e}", "ERROR")
    sys.exit(1)

# --- Step 1: Clone/Update the Repository ---
repo_path = '/content/AnxLight'
if not os.path.exists(repo_path):
    log_message(f"Cloning AnxLight repository to {repo_path}...")
    try:
        result = subprocess.run(
            ['git', 'clone', 'https://github.com/drf0rk/AnxLight.git', repo_path],
            check=True, capture_output=True, text=True, timeout=120
        )
        log_message(f"Clone successful.")
    except subprocess.TimeoutExpired:
        log_message("Git clone operation timed out after 120 seconds", "ERROR")
        sys.exit(1)
    except subprocess.CalledProcessError as e:
        log_message(f"FATAL: Failed to clone repository. Error: {e.stderr}", "ERROR")
        sys.exit(1)
else:
    log_message("AnxLight repository already exists. Pulling latest changes...")
    try:
        subprocess.run(['git', '-C', repo_path, 'reset', '--hard', 'origin/main'], check=True, capture_output=True, text=True, timeout=60)
        pull_result = subprocess.run(['git', '-C', repo_path, 'pull'], check=True, capture_output=True, text=True, timeout=60)
        log_message(f"Pull result: {pull_result.stdout.strip()}")
    except subprocess.TimeoutExpired:
        log_message("Git pull operation timed out. Continuing with existing state.", "WARNING")
    except subprocess.CalledProcessError as e:
        log_message(f"WARNING: Failed to pull latest changes: {e.stderr}", "WARNING")

# --- Step 2: Set Environment ---
os.chdir(repo_path)
log_message(f"Current working directory: {os.getcwd()}")
os.environ['PROJECT_ROOT'] = repo_path
log_message(f"PROJECT_ROOT set to: {os.environ['PROJECT_ROOT']}")

# --- Step 3: Execute the Pre-Flight Setup Script ---
pre_flight_script_path = os.path.join(repo_path, 'scripts', 'pre_flight_setup.py')
python_executable = sys.executable

if not os.path.exists(pre_flight_script_path):
    log_message(f"FATAL: Pre-flight script not found at {pre_flight_script_path}", "ERROR")
    sys.exit(1)

log_message(f"Executing Pre-Flight Setup Script: {pre_flight_script_path}")
log_message(f"All subsequent output will be saved to: {session_log_file}")

try:
    process = subprocess.Popen(
        [python_executable, pre_flight_script_path],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True
    )

    # Stream all output from the subprocess to both the console and the unified log file
    with open(session_log_file, "a") as f:
        for line in iter(process.stdout.readline, ''):
            print(line, end='')
            f.write(line)

    return_code = process.wait()

    if return_code != 0:
        log_message(f"Pre-flight setup script failed with return code {return_code}", "ERROR")
        sys.exit(1)

    log_message("Pre-Flight Setup Script Finished Successfully")

except Exception as e:
    log_message(f"FATAL: An unexpected error occurred during pre-flight setup: {e}", "ERROR")
    import traceback
    log_message(traceback.format_exc(), "ERROR")
    sys.exit(1)

log_message("--- CELL 1: PRE-FLIGHT SETUP END ---")

[2025-06-27 00:54:34] [INFO] --- CELL 1: PRE-FLIGHT SETUP START ---
[2025-06-27 00:54:34] [INFO] Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
[2025-06-27 00:54:34] [INFO] Platform: Linux-6.1.123+-x86_64-with-glibc2.35
[2025-06-27 00:54:34] [INFO] Git version: git version 2.34.1
[2025-06-27 00:54:34] [INFO] Cloning AnxLight repository to /content/AnxLight...
[2025-06-27 00:54:34] [INFO] Clone successful.
[2025-06-27 00:54:34] [INFO] Current working directory: /content/AnxLight
[2025-06-27 00:54:34] [INFO] PROJECT_ROOT set to: /content/AnxLight
[2025-06-27 00:54:34] [INFO] Executing Pre-Flight Setup Script: /content/AnxLight/scripts/pre_flight_setup.py
[2025-06-27 00:54:34] [INFO] All subsequent output will be saved to: /content/anxlight_logs_20250627_005434/session_log_20250627_005434.txt
--- AnxLight Pre-Flight Setup Script v0.1.15 ---
Project Root: /content/AnxLight
Virtual Env Path: /content/AnxLight/anxlight_venv

--- Ensuring essential system packages are inst

In [ ]:
# @title Cell 2: AnxLight v3 Gradio App Launcher (Unified Log)
# This cell launches the main Gradio UI with enhanced process management and unified logging.

import os
import sys
import subprocess
import time
import signal
import atexit
from datetime import datetime

# --- Unified Logging Setup ---
# Use the log file path from Cell 1, or create a new one if run standalone
log_file_from_env = os.environ.get('ANXLIGHT_LOG_FILE')

if log_file_from_env and os.path.exists(os.path.dirname(log_file_from_env)):
    session_log_file = log_file_from_env
else:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = f"/content/anxlight_logs_{timestamp}"
    os.makedirs(log_dir, exist_ok=True)
    session_log_file = f"{log_dir}/session_log_{timestamp}.txt"
    os.environ['ANXLIGHT_LOG_FILE'] = session_log_file
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] [WARNING] Could not find log file from Cell 1. Creating new log at: {session_log_file}")

def log_message(message, level="INFO"):
    """Log a message to both console and the unified session log file"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_msg = f"[{timestamp}] [{level}] {message}"
    print(formatted_msg)
    with open(session_log_file, "a") as f:
        f.write(formatted_msg + "\n")

log_message("--- CELL 2: GRADIO APP LAUNCHER START ---")

# --- Path Verification ---
project_root = os.environ.get('PROJECT_ROOT', '/content/AnxLight')
venv_python_path = os.path.join(project_root, 'anxlight_venv', 'bin', 'python')
main_gradio_script_path = os.path.join(project_root, 'scripts', 'main_gradio_app.py')

if not os.path.exists(venv_python_path):
    log_message(f"CRITICAL ERROR: Virtual environment not found at {venv_python_path}. Please run Cell 1.", "ERROR")
    sys.exit(1)

# --- Process Management ---
gradio_process = None

def cleanup_process():
    global gradio_process
    if gradio_process and gradio_process.poll() is None:
        log_message("Terminating Gradio process...")
        try:
            gradio_process.terminate()
            gradio_process.wait(timeout=5)
            log_message("Gradio process terminated gracefully.")
        except subprocess.TimeoutExpired:
            gradio_process.kill()
            log_message("Gradio process forcefully killed.")
        except Exception as e:
            log_message(f"Error during cleanup: {e}", "ERROR")

atexit.register(cleanup_process)

# --- Launch Gradio App ---
gradio_env = os.environ.copy()
gradio_env['PYTHONPATH'] = f"{os.path.join(project_root, 'modules')}:{project_root}:{os.path.join(project_root, 'scripts')}:{gradio_env.get('PYTHONPATH', '')}"
log_message(f"Launching: {main_gradio_script_path}")
log_message(f"Appending all subsequent output to: {session_log_file}")

try:
    gradio_process = subprocess.Popen(
        [venv_python_path, main_gradio_script_path],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True, env=gradio_env, cwd=project_root
    )
    log_message("Waiting for Gradio URL...")

    url_found = False
    start_time = time.time()
    timeout = 300 # 5 minutes
    url_patterns = ["Running on public URL:", "Public URL:"]

    # Stream all output from the subprocess to both the console and the unified log file
    with open(session_log_file, "a") as f:
        for line in iter(gradio_process.stdout.readline, ''):
            print(line, end='')
            f.write(line)

            if any(p in line for p in url_patterns):
                print("\n" + "="*60)
                print("          GRADIO IS RUNNING AND READY!")
                print("          Click the public URL above to open the UI.")
                print("="*60 + "\n")
                url_found = True
                break

            if time.time() - start_time > timeout:
                log_message(f"Timeout waiting for Gradio URL after {timeout} seconds.", "ERROR")
                break

    if url_found:
        # Continue streaming the rest of the output
        with open(session_log_file, "a") as f:
            for line in iter(gradio_process.stdout.readline, ''):
                print(line, end='')
                f.write(line)
    else:
        log_message("Gradio process finished or timed out before a URL was generated.", "ERROR")

except KeyboardInterrupt:
    log_message("Keyboard interrupt received. Shutting down.", "WARNING")
except Exception as e:
    log_message(f"An error occurred while launching Gradio: {e}", "ERROR")
finally:
    cleanup_process()
    log_message("--- CELL 2: GRADIO APP LAUNCHER END ---")

[2025-06-27 00:56:54] [INFO] --- CELL 2: GRADIO APP LAUNCHER START ---
[2025-06-27 00:56:54] [INFO] Launching: /content/AnxLight/scripts/main_gradio_app.py
[2025-06-27 00:56:54] [INFO] Appending all subsequent output to: /content/anxlight_logs_20250627_005434/session_log_20250627_005434.txt
[2025-06-27 00:56:54] [INFO] Waiting for Gradio URL...
